Preparing necessary imports

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from getpass import getpass
import omero.gateway as gw
import omero

from plots import *
from utils import *

try:
    from credentials import *
except ModuleNotFoundError:
    HOST = input('host:')
    PORT = 4064
    USER = input('user:')
    PASSWORD = getpass('password')
    GROUP = input('group:')

### Defining display functions and a tag_to_function mapping

In [ ]:
def print_map_ann(image):
    print(f'Measurements for image id: {image.getId()}\n{image.getName()}')
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.MapAnnotationI and \
            ann.getNs().startswith('metrics'):
            print('Key\tvalue')
            for m in ann.getValue():
                print(f'{m[0]}\t{m[1]}')

def display_argoSIM_E(images):
    for image in images:
        print_map_ann(image)
        print('Some profiles should go in here')

def display_argoSIM_B(images):
    for image in images:
        print_map_ann(image)
        plot_homogeneity_map(image)
        plot_distances_map(image)
        plot_distances_map_bokeh(image)

def display_psfs(images):
    for image in images:
        plot_psfs(image)



tag_to_function = {'Argo-SIM_E_90deg': display_argoSIM_E,
                   'Argo-SIM_E_0deg': display_argoSIM_E,
                   'Argo-SIM_B': display_argoSIM_B,
                   'PSF beads': display_psfs}

### Open connection

In [ ]:
conn = gw.BlitzGateway(username=USER,
                       passwd=PASSWORD,
                       host=HOST,
                       group=GROUP,
                       port=PORT)

# conn = gw.BlitzGateway(username=input('OMERO username:'), password=getpass('OMERO password:'),
#                        group=GROUP, host=OMERO_SERVER, port=PORT, secure=True)

conn.connect()
print(f'Connection successful: {conn.isConnected()}')

### Selecting the Microscope

In [ ]:
microscopes = list(conn.getObjects('Project'))
micr_menu = widgets.Dropdown(options=[(m.getName(), i) for i, m in enumerate(microscopes)],
                     value=1,
                     description='Select a microscope:',
                     disabled=False,)
display(micr_menu)

### Selecting the Dataset

In [6]:
microscope = microscopes[micr_menu.value]

datasets = list(microscope.listChildren())
dataset_menu = widgets.Dropdown(options=[(d.getName(), i) for i, d in enumerate(datasets)],
                     value=0,
                     description='Select a dataset:',
                     disabled=False,)
display(dataset_menu)

Dropdown(description='Select a dataset:', options=(('2017-02-20_yearly', 0), ('2017-03-20_monthly', 1), ('2017…

### Select the type of analysis to display

In [ ]:
dataset = datasets[dataset_menu.value]
images = list(dataset.listChildren())

# Get the relevant tags associated to the images in the dataset
relevant_tags = ['Argo-SIM_B', 'AArgo-SIM_E_0deg', 'Argo-SIM_E_90deg', 'PSF beads']
present_tags = []

for image in images:
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.TagAnnotationI and \
                ann.getTextValue() in relevant_tags and \
                ann.getTextValue() not in present_tags:
            present_tags.append(ann.getTextValue())

tags_menu = widgets.Dropdown(options=[tag for tag in present_tags],
                             description='Select an analysis type:',
                             value='PSF beads',
                             disabled=False,)
display(tags_menu)

### Run visualization

In [ ]:
images_to_display = []

for image in images:
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.TagAnnotationI and \
                ann.getTextValue() == tags_menu.value:
            images_to_display.append(image)

# table = plot_homogeneity_map(images_to_display[0])
tag_to_function[tags_menu.value](images_to_display)
